In [ ]:
import torch
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DataCollatorForCompletionOnlyLM
import config

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.student_model_name)
response_template_ids = tokenizer("<|im_start|>assistant\n")["input_ids"]
collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)

In [ ]:
dataset = datasets.load_from_disk("/scratch/klambert/dataset/tulu-3-sft-mixture-pretokenized")
logit_dataset = datasets.load_from_disk("/scratch/klambert/slm_ensembles/teacher_logits/teacher_logits")

In [ ]:
print(dataset)

In [ ]:
print(torch.cuda.is_available())

In [ ]:
print(logit_dataset)

In [ ]:

teacher_model = AutoModelForCausalLM.from_pretrained(
    config.teacher_model_name,
    torch_dtype=torch.bfloat16,
)
teacher_model.resize_token_embeddings(new_num_tokens=config.student_vocab_size)
teacher_model.requires_grad_(False)
 

In [ ]:
teacher_eval_results = evaluate_model(dataClass.teacher_model, dataset["test"], collator)

In [ ]:
print(teacher_eval_results["eval_loss"])
print(teacher_eval_results["perplexity"])